In [1]:
%config InlineBackend.figure_formats = {'png', 'retina'}
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

### 다중회귀분석
- 프리미어리그 데이터
- MAE 
    - 득점 : 5.5
    - 실점 : 8.05

In [2]:
# 1. 데이터 로드

In [3]:
with open("./data/premierleague.pkl", "rb") as file:
    datas = pickle.load(file)
df = pd.DataFrame(datas, columns=["name", "gf", "ga", "points"])
df.tail(2)

,name,gf,ga,points
18,Stoke City,35,68,33
19,West Bromwich Albion,31,56,31


In [4]:
# 2. 데이터 전처리 : feature, target

In [5]:
feature = df[["gf", "ga"]]
target = df["points"]

In [6]:
# 3. 모델 학습

In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
%%time
model = LinearRegression().fit(feature, target)

CPU times: user 5.73 ms, sys: 1.64 ms, total: 7.38 ms
Wall time: 8.85 ms


In [9]:
# 4. 모델 평가 : MAE

In [10]:
from sklearn.metrics import mean_absolute_error

In [11]:
# 득점 : 5.5, 실점 : 8.05
pred = np.round(model.predict(feature))
mae = mean_absolute_error(target, pred)
mae

2.9

In [ ]:
# 5. 모델 사용
# 토트넘이 득점을 10점 더 하면 맨유를 앞지를수 있었을까?
# 토트넘 : 77, 맨유 : 81

In [12]:
df.head(3)

,name,gf,ga,points
0,Manchester City,106,27,100
1,Manchester United,68,28,81
2,Tottenham Hotspur,74,36,77


In [13]:
sample_data = feature.loc[2:2]
sample_data = sample_data.reset_index(drop=True)
sample_data["gf"] = 84
# sample_data["ga"] = 31
sample_data

,gf,ga
0,84,36


In [14]:
np.round(model.predict(sample_data))

array([81.])

In [ ]:
# 회귀계수확인 : y = 56.9 + 0.57 * x1 - 0.67 * x2

In [15]:
model.intercept_, model.coef_

(56.91950403942755, array([ 0.57788693, -0.67355499]))

In [ ]:
# 6. 모델 저장 : RAM > HD

In [16]:
# 모델 저장
with open("./data/model.pkl", "wb") as file:
    pickle.dump(model, file)

In [18]:
%ls data

advertising.csv          model.pkl                thermal_power_plant.csv
advertising.pkl          premierleague.csv
credit.csv               premierleague.pkl


In [19]:
# 모델 로드
with open("./data/model.pkl", "rb") as file:
    load_model = pickle.load(file)

In [20]:
# 로드 모델 사용
np.round(load_model.predict(sample_data))

array([81.])

In [ ]:
# 7. statsmodels 패키지 사용 : 회귀분석 요약표 확인
# statsmodels : 더 자세하고 디테일하게 설정 및 모델 학습 결과 요약표를 확인 : 코드 사용이 조금 더 복잡
# sklearn : 코드가 간단 : 요약표와 같은 기능을 제공 X

In [21]:
import statsmodels.api as sm

In [22]:
df = pd.read_csv("./data/premierleague.csv")
df.tail(2)

,name,gf,ga,points
18,Stoke City,35,68,33
19,West Bromwich Albion,31,56,31


In [23]:
# 데이터 전처리 : feature에 상수항 추가
feature = df[["gf","ga"]]
target = df["points"]
sm_feature = sm.add_constant(feature) 
sm_feature.tail(2)

,const,gf,ga
18,1.0,35,68
19,1.0,31,56


In [ ]:
# 모델 학습

In [24]:
model = sm.OLS(target, sm_feature).fit()

In [ ]:
# 요약표 확인

In [25]:
# y = 56.9 + 0.57 * x1 - 0.67 * x2
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     278.4
Date:                Wed, 25 May 2022   Prob (F-statistic):           1.02e-13
Time:                        16:28:06   Log-Likelihood:                -51.744
No. Observations:                  20   AIC:                             109.5
Df Residuals:                      17   BIC:                             112.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         56.9195      6.642      8.569      0.0

In [ ]:
# 모델 사용

In [26]:
model.params

const    56.919504
gf        0.577887
ga       -0.673555
dtype: float64

In [27]:
pred = np.round(np.dot(sm_feature, model.params))
pred[:5]

array([100.,  77.,  75.,  80.,  67.])

In [28]:
mean_absolute_error(target, pred)

2.9